In [1]:
import numpy as np
import torch

from torch.utils.data import DataLoader, random_split

from data_loaders.pulja_data_loader import PuljaDataLoader
from models._20220520_00 import UserModel
from models.utils import collate_fn

if torch.cuda.is_available():
    from torch.cuda import FloatTensor, LongTensor
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
else:
    from torch import FloatTensor, LongTensor

In [2]:
batch_size = 256
seq_len = 100
train_ratio = 0.9

In [3]:
dataset = PuljaDataLoader(seq_len)

In [4]:
dataset.num_c, dataset.num_d

(37, 7)

In [5]:
dataset.c_seqs[0].shape

(101,)

In [6]:
train_size = int(len(dataset) * train_ratio)
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(
    dataset, [train_size, test_size]
)

train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    collate_fn=collate_fn
)
test_loader = DataLoader(
    test_dataset, batch_size=test_size, shuffle=True,
    collate_fn=collate_fn
)

In [7]:
model = UserModel(dataset.num_c, dataset.num_d, 50)

In [8]:
model

UserModel(
  (D1): Embedding(7, 1)
  (D2): Embedding(7, 50)
  (gru): GRU(100, 50, batch_first=True)
  (linear_1): Sequential(
    (0): Linear(in_features=50, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
  (linear_2): Sequential(
    (0): Linear(in_features=150, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [9]:
for data in train_loader:
    print(data)
    
    c_seq, d_seq, r_seq, cshft_seq, dshft_seq, rshft_seq, m_seq = data
    
    print(model(c_seq, d_seq, r_seq))
    
    break

(tensor([[ 0.,  0.,  0.,  ..., -0., -0., -0.],
        [24., 24., 24.,  ..., 35., 35., 35.],
        [ 2.,  2.,  2.,  ..., -0., -0., -0.],
        ...,
        [28., 28., 22.,  ..., 17., 17., 17.],
        [26., 26., 26.,  ..., 30., 30., 30.],
        [21., 21., 21.,  ...,  9.,  9.,  9.]]), tensor([[0., 0., 0.,  ..., -0., -0., -0.],
        [2., 2., 2.,  ..., 1., 2., 2.],
        [2., 2., 3.,  ..., -0., -0., -0.],
        ...,
        [2., 2., 4.,  ..., 3., 3., 5.],
        [1., 1., 1.,  ..., 2., 2., 2.],
        [2., 0., 1.,  ..., 1., 0., 1.]]), tensor([[0.0000, 0.0000, 0.0000,  ..., -0.0000, -0.0000, -0.0000],
        [0.0000, 0.1765, 0.0000,  ..., 0.9020, 0.0000, 0.0000],
        [1.0000, 0.0000, 0.3268,  ..., -0.0000, -0.0000, -0.0000],
        ...,
        [1.0000, 1.0000, 0.8706,  ..., 0.0000, 0.4052, 0.0000],
        [0.0000, 0.9281, 0.0392,  ..., 0.0000, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.5621, 0.0000, 0.0000]]), tensor([[ 0.,  0.,  0.,  ..., -0., -0., -0

RuntimeError: Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [ ]:
model(LongTensor(dataset.c_seqs[:5]), LongTensor(dataset.d_seqs[:5]), FloatTensor(dataset.r_seqs[:5]))

In [ ]:
alpha_seq, beta_seq, gamma_seq, h_seq, C_seq = model(LongTensor(dataset.c_seqs[:5]), LongTensor(dataset.d_seqs[:5]), FloatTensor(dataset.r_seqs[:5]))

In [ ]:
alpha_seq.shape